# Data Analysis Agent

##### In this notebook, we will create an agent that given a dataframe will answer user questions about it. This example is based on the [Simple Data Analysis Agent](https://github.com/NirDiamant/GenAI_Agents/blob/main/all_agents_tutorials/simple_data_analysis_agent_notebook-pydanticai.ipynb) from the [GenAI Agents](https://github.com/NirDiamant/GenAI_Agents) repository.

In [1]:
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import gradio as gr

from typing import List,Annotated
from pydantic import Field

from agente.core.base import BaseAgent
from agente.core.decorators import function_tool

from dotenv import load_dotenv
## Load and set environment variables from .env file
load_dotenv()


/Users/miguelwon/opt/miniconda3/envs/agente/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### We start by simulating a dataset of car sales

In [2]:
# Generate sample data
n_rows = 1000

# Generate dates
start_date = datetime(2022, 1, 1)
dates = [start_date + timedelta(days=i) for i in range(n_rows)]

# Define data categories
makes = ['Toyota', 'Honda', 'Ford', 'Chevrolet', 'Nissan', 'BMW', 'Mercedes', 'Audi', 'Hyundai', 'Kia']
models = ['Sedan', 'SUV', 'Truck', 'Hatchback', 'Coupe', 'Van']
colors = ['Red', 'Blue', 'Black', 'White', 'Silver', 'Gray', 'Green']

# Create the dataset
data = {
    'Date': dates,
    'Make': np.random.choice(makes, n_rows),
    'Model': np.random.choice(models, n_rows),
    'Color': np.random.choice(colors, n_rows),
    'Year': np.random.randint(2015, 2023, n_rows),
    'Price': np.random.uniform(20000, 80000, n_rows).round(2),
    'Mileage': np.random.uniform(0, 100000, n_rows).round(0),
    'EngineSize': np.random.choice([1.6, 2.0, 2.5, 3.0, 3.5, 4.0], n_rows),
    'FuelEfficiency': np.random.uniform(20, 40, n_rows).round(1),
    'SalesPerson': np.random.choice(['Alice', 'Bob', 'Charlie', 'David', 'Eva'], n_rows)
}

# Create DataFrame and sort by date
df = pd.DataFrame(data).sort_values('Date')
df

,Date,Make,Model,Color,Year,Price,Mileage,EngineSize,FuelEfficiency,SalesPerson
0,2022-01-01,Ford,SUV,Gray,2020,23887.50,49176.0,3.0,38.2,Charlie
1,2022-01-02,Audi,Van,White,2017,28332.15,11629.0,4.0,29.9,Charlie
2,2022-01-03,Audi,Truck,White,2022,61301.76,63298.0,3.5,37.1,David
3,2022-01-04,Ford,SUV,White,2016,31821.71,40040.0,4.0,32.9,Bob
4,2022-01-05,Hyundai,SUV,Silver,2018,38096.93,10091.0,1.6,30.4,Bob
...,...,...,...,...,...,...,...,...,...,...
995,2024-09-22,Chevrolet,Van,Blue,2019,31217.87,40317.0,3.0,23.9,Bob
996,2024-09-23,Toyota,Sedan,Silver,2021,64381.95,48529.0,3.5,30.7,Alice
997,2024-09-24,Ford,Sedan,Red,2019,45251.72,35962.0,3.5,34.2,David
998,2024-09-25,Kia,Sedan,Red,2015,62439.69,14491.0,1.6,22.4,Bob


In [3]:
print(df.head().to_markdown())

|    | Date                | Make    | Model   | Color   |   Year |   Price |   Mileage |   EngineSize |   FuelEfficiency | SalesPerson   |
|---:|:--------------------|:--------|:--------|:--------|-------:|--------:|----------:|-------------:|-----------------:|:--------------|
|  0 | 2022-01-01 00:00:00 | Ford    | SUV     | Gray    |   2020 | 23887.5 |     49176 |          3   |             38.2 | Charlie       |
|  1 | 2022-01-02 00:00:00 | Audi    | Van     | White   |   2017 | 28332.2 |     11629 |          4   |             29.9 | Charlie       |
|  2 | 2022-01-03 00:00:00 | Audi    | Truck   | White   |   2022 | 61301.8 |     63298 |          3.5 |             37.1 | David         |
|  3 | 2022-01-04 00:00:00 | Ford    | SUV     | White   |   2016 | 31821.7 |     40040 |          4   |             32.9 | Bob           |
|  4 | 2022-01-05 00:00:00 | Hyundai | SUV     | Silver  |   2018 | 38096.9 |     10091 |          1.6 |             30.4 | Bob           |


In [4]:
SYSTEM_PROMPT="""You are an AI assistant that helps extract information from a pandas DataFrame.
If asked about columns, be sure to check the column names first.
Be concise in your answers."""


class SimpleAnalysisAgent(BaseAgent):
    agent_name: str = "SimpleAnalysisAgent"
    system_prompt:str = SYSTEM_PROMPT
    df: pd.DataFrame = Field(default_factory=pd.DataFrame) 

    completion_kwargs: dict = {
        "model": "claude-sonnet-4-20250514",
        "stream": False,
    }

    @function_tool
    async def df_query(self,query: str):
        """A tool for running queries on the `pandas.DataFrame`. Use this tool to interact with the DataFrame.

        `query` will be executed using `pd.eval(query, target=df)`, so it must contain syntax compatible with
        `pandas.eval`.

        Args:
            query: The query to run on the DataFrame.
        """
        # Print the query for debugging purposes and fun :)
        print(f'Running query: `{query}`')
        return str(pd.eval(query, target=self.df))

In [5]:
agent = SimpleAnalysisAgent()
agent.add_message( role = "user", content = "What are the column names in this dataset?")

responses = await agent.run()

Executing agent: SimpleAnalysisAgent
Executing tool: df_query (agent: SimpleAnalysisAgent)
Running query: `df.columns.tolist()`
Executing agent: SimpleAnalysisAgent


In [6]:
responses[-1].dict()

/var/folders/zq/0fkhk67d5f94s7fqjygx4xfm0000gn/T/ipykernel_70815/1492498605.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  responses[-1].dict()


{'id': 'chatcmpl-051aa54e-869a-42b9-b588-3719ea8bd2df',
 'created': 1758962680,
 'model': 'claude-sonnet-4-20250514',
 'object': 'chat.completion',
 'system_fingerprint': None,
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': 'The dataset contains the following column names:\n\n1. Date\n2. Make\n3. Model\n4. Color\n5. Year\n6. Price\n7. Mileage\n8. EngineSize\n9. FuelEfficiency\n10. SalesPerson',
    'role': 'assistant',
    'tool_calls': None,
    'function_call': None}}],
 'usage': {'completion_tokens': 63,
  'prompt_tokens': 623,
  'total_tokens': 686,
  'completion_tokens_details': None,
  'prompt_tokens_details': {'audio_tokens': None,
   'cached_tokens': 0,
   'text_tokens': None,
   'image_tokens': None},
  'cache_creation_input_tokens': 0,
  'cache_read_input_tokens': 0}}

In [7]:
agent.add_message("user","What is the most sold brand?")
responses = await agent.run()

Executing agent: SimpleAnalysisAgent
Executing tool: df_query (agent: SimpleAnalysisAgent)
Running query: `df['Make'].value_counts().head(1)`
Executing agent: SimpleAnalysisAgent


In [8]:
for m in agent.conv_history.messages:
    print(m)
    print("--------------------------------")

role='system' agent_name='SimpleAnalysisAgent' content=[Content(type='text', text='You are an AI assistant that helps extract information from a pandas DataFrame.\nIf asked about columns, be sure to check the column names first.\nBe concise in your answers.')] tool_calls=None tool_call_id=None tool_name=None hidden=False id=None usage=None timestamp=datetime.datetime(2025, 9, 27, 8, 44, 36, 639493)
--------------------------------
role='user' agent_name='SimpleAnalysisAgent' content=[Content(type='text', text='What are the column names in this dataset?')] tool_calls=None tool_call_id=None tool_name=None hidden=False id=None usage=None timestamp=datetime.datetime(2025, 9, 27, 8, 44, 36, 639570)
--------------------------------
role='assistant' agent_name='SimpleAnalysisAgent' content=[Content(type='text', text="I'll check the column names in the dataset for you.")] tool_calls=None tool_call_id=None tool_name=None hidden=False id=None usage=Usage(completion_tokens=72, prompt_tokens=487, 

In [9]:
responses[-1].dict()

/var/folders/zq/0fkhk67d5f94s7fqjygx4xfm0000gn/T/ipykernel_70815/1492498605.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  responses[-1].dict()


{'id': 'chatcmpl-04f05b65-8459-48fc-a6de-535904878594',
 'created': 1758962685,
 'model': 'claude-sonnet-4-20250514',
 'object': 'chat.completion',
 'system_fingerprint': None,
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': 'The most sold brand is **Kia** with 130 sales.',
    'role': 'assistant',
    'tool_calls': None,
    'function_call': None}}],
 'usage': {'completion_tokens': 19,
  'prompt_tokens': 808,
  'total_tokens': 827,
  'completion_tokens_details': None,
  'prompt_tokens_details': {'audio_tokens': None,
   'cached_tokens': 0,
   'text_tokens': None,
   'image_tokens': None},
  'cache_creation_input_tokens': 0,
  'cache_read_input_tokens': 0}}

In [10]:
agent.add_message("user","What is the average price of the cars?")
responses = await agent.run()

Executing agent: SimpleAnalysisAgent
Executing tool: df_query (agent: SimpleAnalysisAgent)
Running query: `df['Price'].mean()`
Executing agent: SimpleAnalysisAgent


In [11]:
agent.conv_history.messages[-1].dict()

/var/folders/zq/0fkhk67d5f94s7fqjygx4xfm0000gn/T/ipykernel_70815/457905648.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  agent.conv_history.messages[-1].dict()


{'role': 'assistant',
 'agent_name': 'SimpleAnalysisAgent',
 'content': [{'type': 'text',
   'text': 'The average price of the cars is **$48,983.31**.'}],
 'tool_calls': None,
 'tool_call_id': None,
 'tool_name': None,
 'hidden': False,
 'id': None,
 'usage': {'completion_tokens': 18, 'prompt_tokens': 929, 'total_tokens': 947},
 'timestamp': datetime.datetime(2025, 9, 27, 8, 44, 57, 122534)}

## Streaming

### Now the same but with streaming with Gradio

In [14]:
import uuid

class SimpleAnalysisAgent(BaseAgent):
    agent_name: str = "SimpleAnalysisAgent"
    df: pd.DataFrame = df
    completion_kwargs: dict = {
        "model": "gpt-4o",
        "stream": False,
    }

    @function_tool
    async def df_query(self,query: Annotated[str, "The query to run on the DataFrame."]) -> str:
        """A tool for running queries on the `pandas.DataFrame`. Use this tool to interact with the DataFrame.

        `query` will be executed using `pd.eval(query, target=df)`, so it must contain syntax compatible with
        `pandas.eval`.
        """
        # Print the query for debugging purposes and fun :)
        print(f'Running query: `{query}`')
        return str(pd.eval(query, target=self.df))


In [ ]:
from litellm import ModelResponseStream
from agente.models.schemas import StreamResponse
import gradio as gr


def get_new_agent():
    """Create a fresh agent instance"""
    new_agent = SimpleAnalysisAgent()
    new_agent.add_message("system", SYSTEM_PROMPT)
    return new_agent

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    # Initialize with a function call instead of direct instantiation
    agent_state = gr.State(value=None)

    def user(user_message, agent):
        if agent is None:
            agent = get_new_agent()
        agent.add_message("user", user_message)
        return "", [{"role": "user", "content": user_message}], agent

    async def bot(history, agent):
        if not history:
            yield [], agent
            return
        
        history.append({"role": "assistant", "content": ""})
        
        response_generator = await agent.run(stream=True)
        async for chunk in response_generator:
            #check if chunk is litellm or agente
            if isinstance(chunk, ModelResponseStream):
                if chunk.choices[0].delta.content:
                    history[-1]["content"] += chunk.choices[0].delta.content
                    yield history, agent
            elif isinstance(chunk, StreamResponse):
                if chunk.content:
                    history[-1]["content"] += chunk.content
                    yield history, agent
            else:
                raise ValueError(f"Unknown chunk type: {type(chunk)}")


    def reset_state():
        return None, get_new_agent()

    msg.submit(user, [msg, agent_state], [msg, chatbot, agent_state], queue=False).then(
        bot, [chatbot, agent_state], [chatbot, agent_state]
    )
    clear.click(reset_state, None, [chatbot, agent_state], queue=False)

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Executing agent: SimpleAnalysisAgent
Executing agent: SimpleAnalysisAgent
Executing tool: df_query (agent: SimpleAnalysisAgent)
Running query: `columns`
Tool error in df_query: Error in tool 'df_query': name 'columns' is not defined
Executing agent: SimpleAnalysisAgent
Executing tool: df_query (agent: SimpleAnalysisAgent)
Running query: `df.columns`
Executing agent: SimpleAnalysisAgent
Executing tool: df_query (agent: SimpleAnalysisAgent)
Running query: `Make.value_counts().idxmax()`
Tool error in df_query: Error in tool 'df_query': name 'Make' is not defined
Executing agent: SimpleAnalysisAgent
Executing tool: df_query (agent: SimpleAnalysisAgent)
Running query: `df['Make'].value_counts().idxmax()`
Executing agent: SimpleAnalysisAgent
